In [1]:
import warnings #suppress warnings
import torch
import random
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from scipy.io import arff
from pyod.models.anogan import AnoGAN
from pyod.models.mo_gaal import MO_GAAL
from pyod.models.lof import LOF
from pyod.models.gmm import GMM

warnings.simplefilter("ignore")

2023-05-14 09:59:26.660063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 09:59:27.122964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class CustomDataset(Dataset):
    def __init__(self, path):
        # start preprocessing 
        self.arff_data = arff.loadarff(path)
        self.df = pd.DataFrame(self.arff_data[0])
        #0 is outlier, 1 is normal data
        self.df["outlier"] = pd.factorize(self.df["outlier"])[0]
        #end preprocessing
        
        self.data_tensor = torch.tensor(self.df.to_numpy()).float()
        self.data_numpy = self.df.to_numpy()
        self.n = self.df.shape[0]
        
    def __len__(self):
        return len(self.data_tensor)
    
    def __getitem__(self, i):
        return self.data_tensor[i]

In [3]:
path = "/home/denis/Documents/Resources_Thesis/Waveform_withoutdupl_norm_v01.arff"
seed = 777
torch.manual_seed(seed)
random.seed(seed)
num_workers = 2
batch_size = 128
#number of used GPUs
gpu = 0 

usedDevice = torch.device("cpu" if gpu == 0 else "cuda")
dataset = CustomDataset(path)
train_set, eval_set, test_set = torch.utils.data.random_split(dataset.data_numpy[:,:-1], [0.6,0.2,0.2]) #PFUSCH WEGEN NUMPY?
#maybe data loader for each category?
dataloader = DataLoader(dataset=dataset.data_tensor, batch_size = batch_size, shuffle=True, num_workers=num_workers)

In [4]:
def check_accuracy(prediction):
    correct = 0
    wrong = 0
    idx = 0
    for i in test_set.indices:
        if dataset.data_numpy[i][-1] != prediction[idx]:
            correct += 1
        else: 
            wrong += 1
        idx += 1

    print(len(test_set))
    print("Accuracy: " + str(correct/len(test_set)))

In [6]:
mogaal_model = MO_GAAL()
mogaal_model.fit(train_set)

2023-05-14 09:59:45.141298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-14 09:59:45.155614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-14 09:59:45.155756: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1 of 60

Testing for epoch 1 index 1:


2023-05-14 09:59:46.499450: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


 1/65 [..............................] - ETA: 1s

2023-05-14 09:59:46.871358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1000]
	 [[{{node Placeholder/_1}}]]


...

2023-05-14 10:02:21.124714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1000]
	 [[{{node Placeholder/_1}}]]


16/16 [==============================] - 0s 851us/step - loss: 4.7545

Testing for epoch 60 index 4:
16/16 [==============================] - 0s 718us/step - loss: 4.7752


2023-05-14 10:02:21.881594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1000]
	 [[{{node Placeholder/_1}}]]


65/65 [==============================] - 0s 487us/step


MO_GAAL(contamination=0.1, k=10, lr_d=0.01, lr_g=0.0001, momentum=0.9,
    stop_epochs=20)

In [7]:
test_mogaal_pred = mogaal_model.predict(test_set)

22/22 [==============================] - 0s 579us/step


In [8]:
check_accuracy(test_mogaal_pred)

688
Accuracy: 0.9563953488372093


In [9]:
anogan_model = AnoGAN()
anogan_model.fit(train_set)

AnoGAN(D_layers=[20, 10, 5], G_layers=[20, 10, 3, 10, 20],
    activation_hidden='tanh', batch_size=32, contamination=0.1,
    dropout_rate=0.2, epochs=500, epochs_query=20,
    index_D_layer_for_recon_error=1, latent_dim_G=2, learning_rate=0.001,
    learning_rate_query=0.01, output_activation=None, preprocessing=False,
    verbose=0)

In [10]:
test_anogan_pred = anogan_model.predict(test_set)

In [11]:
check_accuracy(test_anogan_pred)

688
Accuracy: 0.8648255813953488


In [12]:
lof_model = LOF()
lof_model.fit(train_set)

LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2)

In [13]:
test_lof_pred = lof_model.predict(test_set)

In [14]:
check_accuracy(test_lof_pred)

688
Accuracy: 0.8241279069767442


In [30]:
gmm_model = GMM(n_components=1)
gmm_model.fit(train_set)

GMM(contamination=0.1, covariance_type='full', init_params='kmeans',
  max_iter=100, means_init=None, n_components=1, n_init=1,
  precisions_init=None, random_state=None, reg_covar=1e-06, tol=0.001,
  warm_start=False, weights_init=None)

In [31]:
test_gmm_pred = gmm_model.predict(test_set)

In [32]:
check_accuracy(test_gmm_pred)

688
Accuracy: 0.8561046511627907
